<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/low_level/oss_ingestion_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building RAG from Scratch (Open-source only!)

In this tutorial, we show you how to build a data ingestion pipeline into a vector database, and then build a retrieval pipeline from that vector database, from scratch.

Notably, we use a fully open-source stack:

- Sentence Transformers as the embedding model
- Elasticsearch as the vector store (we support many other [vector stores](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html) too!)
- Llama 2 as the LLM (through [llama.cpp](https://github.com/ggerganov/llama.cpp))

## Setup

We setup our open-source components.
1. Sentence Transformers
2. Llama 2
3. We initialize postgres and wrap it with our wrappers/abstractions.

#### Sentence Transformers

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp
%pip install llama-index-vector-stores-elasticsearch
%pip install tqdm

In [ ]:
!pip install llama-index

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Llama CPP

In this notebook, we use the [`llama-2-chat-7b-gguf`](https://huggingface.co/TheBloke/Llama-2-7B-chat-GGUF) model, along with the proper prompt formatting.

Check out our [Llama CPP guide](https://gpt-index.readthedocs.io/en/stable/examples/llm/llama_2_llama_cpp.html) for full setup instructions/details.

In [ ]:
!pip install llama-cpp-python


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [37]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_path = "llama-2-7b-chat.Q3_K_S.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=512,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4000,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 0},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

#### Initialize Elasticsearch

Using an existing elasticsearch running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html).

Use this command in the terminal to launch a docker hosted Elasticsearch container

```
docker run -p 9200:9200 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "xpack.security.http.ssl.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  docker.elastic.co/elasticsearch/elasticsearch:8.9.0
```

In [21]:
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [4]:
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

vector_store = ElasticsearchStore(
    index_name="qna_mental_health",
    es_url="http://localhost:9200",
)

## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

We fast-track the steps here (can skip metadata extraction). More details can be found [in our dedicated ingestion guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/ingestion.html).

### 1. Load Data

In [5]:
file_path_test="counsel-chat-best-answer-test.csv"
file_path_train="counsel-chat-best-answer-train.csv"

import pandas as pd

df_test = pd.read_csv(file_path_test)
df_train = pd.read_csv(file_path_train)

### 2. Use a Text Splitter to Split Documents

In [6]:
def combined_data(data):
    return {
        "question": data['questionText'],
        "answer": data['answerText'],
        "topic": data['topic']
    }

df_train['qa_mental_health'] = df_train.apply(combined_data, axis=1)

In [7]:
df_train['qa_mental_health'][0]

{'question': "I have been diagnosed with general anxiety and depression by my family doctor. They wrote a prescription for me to have an emotional support dog, I have the paper work, and I gave it to my apartment manager. They said I can't keep the ESD because I'm not disabled. What do you suggest I do?",
 'answer': 'This can be a difficult situation. \xa0Typically, only animals that are specifically trains to accomplish a specific task are legally protected as Service Animsls. Even though that can be very helpful, emotional support animals are not generally protected in the same way.You might not be able to make your landlord accommodate you. If possible, you may want to consider a different apparent that is more animal friendly.',
 'topic': 'depression'}

In [8]:
df_final = pd.DataFrame(df_train['qa_mental_health'])
df_final['question_title'] = df_train['questionTitle']

In [9]:
df_final.head()

,qa_mental_health,question_title
0,{'question': 'I have been diagnosed with gener...,My apartment manager won't let me keep an emot...
1,{'question': 'There are many people willing to...,Why do I feel like I don't belong anywhere?
2,{'question': 'My girlfriend just quit drinking...,How can I help my girlfriend?
3,{'question': 'I don't know how else to explain...,How do I stop feeling empty?
4,{'question': 'I tried telling my husband I was...,How can I get my husband to listen to my needs...


### 3. Manually Construct Nodes from Text Chunks

In [10]:
from llama_index.core.schema import TextNode

nodes = []
for ans, title in zip(df_final['qa_mental_health'], df_final['question_title']):
    node = TextNode(
        text=str(ans),
    )
    node.metadata = {"title": title}
    nodes.append(node)

### 4. Generate Embeddings for each Node

Here we generate embeddings for each Node using a sentence_transformers model.

In [11]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

### 5. Load Nodes into a Vector Store

We now insert these nodes into our `ElasticsearchVectorStore`.

In [12]:
vector_store.add(nodes)

['8bb12a5e-ff00-4548-9ad3-6452376f7490',
 '19c84223-b0af-499f-91ad-d02e6b668572',
 '9392cbb0-52e1-4cec-8f24-c1c22133dbfb',
 '9f114635-e505-479f-bdd8-9ce5bdc961d9',
 '3538156f-3fe0-4b7a-9128-229b341d0e8d',
 '055e1e8a-8da3-4bd7-8f14-059bfb456545',
 '42328b14-7b85-4f72-8ae3-6b5c2cb6f2de',
 'd8950f34-c10a-45e8-89e2-9af6bff772ef',
 'ce653213-2c85-4883-a6b9-8cef3e24243e',
 'a9c9d511-8f0f-4eb0-ac15-5e6c1a95e257',
 'ff5e121a-d325-45fe-9439-d4e93236b14f',
 '108c81de-b6c0-48db-a449-0e9ba63c2103',
 '88c004cf-1ea0-4f39-940b-766a31130045',
 '22efc545-66ec-4311-b1a2-f262b41132df',
 '767e8553-1cf7-4286-9c73-a8060b783e51',
 '0607ebfa-3c6b-4d29-bf33-c8a57d8bbb4a',
 'd830b38d-4a67-464b-a83d-f66461ae61e0',
 'e788d21c-e694-41ff-a1fd-08b298f69811',
 '316fad1f-930d-434e-9af2-3d8528c67f32',
 '9fc6dbd9-4e38-44c0-a804-e38b832aa8cd',
 '0dcc145b-f50e-471b-8f76-42a968369df2',
 '039a652f-a2b7-4f18-a2f0-511f6013916e',
 'da315144-5ae7-4eb5-8388-9c394fcd7cac',
 'fd4f2465-b0df-461c-89f7-94ac0cd3eeaf',
 'fe8363b6-12dd-

## Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at our [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [17]:
query_str = """I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. 
    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. 
    I’ve never had counseling about any of this. Do I have too many issues to address in counseling?
    """

### 1. Generate a Query Embedding

In [18]:
query_embedding = embed_model.get_query_embedding(query_str)

### 2. Query the Vector Database

In [19]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [20]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

{'question': 'I stress over everything. If I don\'t have enough "quality time" with my boyfriend, I start to feel resentment towards him. He has three children, and they are great kids, but I find we don\'t have much time together. I break down easily and find myself depressed.', 'answer': 'Everyone has some level of anxiety - it\'s what helps us respond to stressors in our lives and clues us into the fact that we need to respond to something going on. However, if you\'re feeling overwhelmed by racing thoughts, feeling like you spend a lot of energy worrying about something specific or even pretty much anything at all, and you\'re starting to find that it\'s getting in your way when it comes to living your life the way you want, then I\'d suggest seeing a counselor or therapist for an assessment for anxiety.\xa0Your other concerns, though, seem pretty "normal" for someone who is in a relationship with a partner who has children. As a married stepmother, I\'ve been there, and as a thera

### 3. Parse Result into a Set of Nodes

In [21]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

### 4. Put into a Retriever

In [22]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: ElasticsearchStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [23]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

## Plug this into our RetrieverQueryEngine to synthesize a response

In [38]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [39]:
query_str = """I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. 
    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. 
    I’ve never had counseling about any of this. Do I have too many issues to address in counseling?
    """

response = query_engine.query(query_str)


llama_print_timings:        load time =   16833.79 ms
llama_print_timings:      sample time =      22.89 ms /   237 runs   (    0.10 ms per token, 10352.96 tokens per second)
llama_print_timings: prompt eval time =   31449.02 ms /   913 tokens (   34.45 ms per token,    29.03 tokens per second)
llama_print_timings:        eval time =   34101.34 ms /   236 runs   (  144.50 ms per token,     6.92 tokens per second)
llama_print_timings:       total time =   65948.91 ms /  1149 tokens


In [40]:
print(str(response))

 No, you don't have too many issues to address in counseling. It's completely normal and healthy to want to work through your past experiences, mental health struggles, and relationship concerns with a trained therapist or counselor. In fact, seeking professional help is often the first step towards healing and growth.
It's important to remember that you don't have to go through any of this alone. A therapist can provide a safe and supportive space for you to process your experiences and emotions, and offer tools and strategies to help you manage your mental health and relationships.
Additionally, it's important to find a therapist who is experienced in working with survivors of sexual abuse and individuals with low self-esteem. They can provide specialized support and guidance to help you work through these issues.
Overall, don't hesitate to seek professional help if you feel like you're struggling to cope with your experiences or mental health. It's okay to ask for help, and it's imp

In [41]:
print(response.source_nodes[0].get_content())

{'question': 'I stress over everything. If I don\'t have enough "quality time" with my boyfriend, I start to feel resentment towards him. He has three children, and they are great kids, but I find we don\'t have much time together. I break down easily and find myself depressed.', 'answer': 'Everyone has some level of anxiety - it\'s what helps us respond to stressors in our lives and clues us into the fact that we need to respond to something going on. However, if you\'re feeling overwhelmed by racing thoughts, feeling like you spend a lot of energy worrying about something specific or even pretty much anything at all, and you\'re starting to find that it\'s getting in your way when it comes to living your life the way you want, then I\'d suggest seeing a counselor or therapist for an assessment for anxiety.\xa0Your other concerns, though, seem pretty "normal" for someone who is in a relationship with a partner who has children. As a married stepmother, I\'ve been there, and as a thera

## Get output of all test questions 

In [51]:
df_test = pd.read_csv(file_path_test)

responses = []

for index, row in df_test.iterrows():
    query_str = row['questionText']
    response = query_engine.query(query_str)
    responses.append(str(response))
    print (str(index+1) + "/" + str(len(df_test)) + " done")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16833.79 ms
llama_print_timings:      sample time =      34.19 ms /   350 runs   (    0.10 ms per token, 10236.31 tokens per second)
llama_print_timings: prompt eval time =  112741.02 ms /     2 tokens (56370.51 ms per token,     0.02 tokens per second)
llama_print_timings:        eval time =  108006.37 ms /   349 runs   (  309.47 ms per token,     3.23 tokens per second)
llama_print_timings:       total time =  114315.10 ms /   351 tokens


1/112 done


Llama.generate: prefix-match hit

llama_print_timings:        load time =   16833.79 ms
llama_print_timings:      sample time =      53.47 ms /   416 runs   (    0.13 ms per token,  7780.21 tokens per second)
llama_print_timings: prompt eval time =   54146.47 ms /  1157 tokens (   46.80 ms per token,    21.37 tokens per second)
llama_print_timings:        eval time =  369499.77 ms /   415 runs   (  890.36 ms per token,     1.12 tokens per second)
llama_print_timings:       total time =  424834.27 ms /  1572 tokens


2/112 done


Llama.generate: prefix-match hit

llama_print_timings:        load time =   16833.79 ms
llama_print_timings:      sample time =      59.87 ms /   512 runs   (    0.12 ms per token,  8551.72 tokens per second)
llama_print_timings: prompt eval time =   36736.64 ms /   733 tokens (   50.12 ms per token,    19.95 tokens per second)
llama_print_timings:        eval time =  141932.67 ms /   511 runs   (  277.75 ms per token,     3.60 tokens per second)
llama_print_timings:       total time =  179868.94 ms /  1244 tokens


...

110/112 done


Llama.generate: prefix-match hit

llama_print_timings:        load time =   16833.79 ms
llama_print_timings:      sample time =      44.81 ms /   512 runs   (    0.09 ms per token, 11426.28 tokens per second)
llama_print_timings: prompt eval time =   29086.83 ms /  1049 tokens (   27.73 ms per token,    36.06 tokens per second)
llama_print_timings:        eval time =   63096.14 ms /   511 runs   (  123.48 ms per token,     8.10 tokens per second)
llama_print_timings:       total time =   93116.11 ms /  1560 tokens


111/112 done


Llama.generate: prefix-match hit

llama_print_timings:        load time =   16833.79 ms
llama_print_timings:      sample time =      10.17 ms /   105 runs   (    0.10 ms per token, 10324.48 tokens per second)
llama_print_timings: prompt eval time =   12429.16 ms /   455 tokens (   27.32 ms per token,    36.61 tokens per second)
llama_print_timings:        eval time =   11836.94 ms /   104 runs   (  113.82 ms per token,     8.79 tokens per second)
llama_print_timings:       total time =   24450.93 ms /   559 tokens


112/112 done


In [50]:
len(df_test)

112

In [52]:
df_test['response'] = responses
df_test.to_csv("counsel-chat-best-answer-test-response.csv", index=False)